In [ ]:
import sqlite3
import numpy as np
import csv
import requests
import os
import codecs
import zipfile
import openpyxl
import xlrd ,xlwt

url = "https://data.medicare.gov/views/bg9k-emty/files/0a9879e0-3312-4719-a1db-39fd114890f1?content_type=application%2Fzip%3B%20charset%3Dbinary&filename=Hospital_Revised_Flatfiles.zip"

#saving the url
r = requests.get(url)  
staging_dir_name = "staging"
current_directory = os.getcwd()

#making the staging directory
os.mkdir(staging_dir_name)
zip_file_name = os.path.join(staging_dir_name,"Hospital_Revised_Flatfiles.zip")
zf = open(zip_file_name,'wb')
zf.write(r.content)
zf.close()

#extracting the zipfile
z = zipfile.ZipFile(zip_file_name,'r')
z.extractall(staging_dir_name)    
z.close()

#removing the files other than csv
os.remove(zip_file_name, dir_fd=None)
file_name = os.listdir(staging_dir_name)
file_name.remove('Hospital.pdf')
file_name.remove('readme.txt')
file_name.remove('FY2015_Percent_Change_in_Medicare_Payments.csv')

conn = sqlite3.connect("medicare_hospital_compare.db")
c = conn.cursor()

#encoding into cp1252 format and coneverting to utf-8 by checking the nulls
for file in file_name:
    fn =os.path.join(staging_dir_name,file)
    print(fn)
    in_fp = open(fn, 'rt', encoding = 'cp1252')
    input_data = in_fp.read()
    in_fp.close()

    ofn =os.path.join(staging_dir_name,file)
    out_fp = open(ofn, 'wt', encoding = 'utf-8')
    for item in input_data:
        if item != '\0':
            out_fp.write(item)            
    out_fp.close()

    sql = ''
    sqld =''
    table_name = file
    
    #formatting the table as per requirement
    
    for ch in [" ", "-","/"]:
        table_name = table_name.replace(ch , "_")
    if (table_name[0].isalpha()!= True):
        table_name = 't_' + table_name
    tablename = os.path.splitext(os.path.basename(table_name))[0]
    tablename = tablename.lower()
    #print(tablename)
    file_path = os.path.join(staging_dir_name,file)
    with open(file_path, encoding = 'utf-8') as fp:
        reader = csv.reader(fp, delimiter = ',')
        headers = next(reader)
                
                    
        sql = "CREATE TABLE IF NOT EXISTS " + tablename +' ( '#creating table
        
        headers = [elements.lower() for elements in headers]
        header_chng =[]
        for header in headers:
            for ch in ["%"]:
                header = header.replace(ch , "pct")
            if (header[0].isalpha()!= True):
                #checking for correct name
                header = 'c_' + header
            for ch in ["/"," ",",","-"]:
                header = header.replace(ch , "_")#getting the name right
            header_chng.append(header)   
            sql += header+ ' text, '
        sql = sql[:-2]#executing 
        sql += ")"
        print(sql)
        sqld = "DROP table IF EXISTS " + tablename
        c.execute(sqld)
        c.execute(sql)
        
        for row in reader:
            if row == [' ']:
                continue
            else:
                insert = 'insert into %s' %tablename + ' (' + ', '.join([head for  head in header_chng]) + ')' + 'values'+'('+','.join(['? ' for head in header_chng]) +')'
                c.execute(insert,tuple(row))
        conn.commit()
        

        
#creating lists
list_focus_state = []
list_element = []
final_list = []
with conn:
    cur = conn.cursor()
    cur.execute("select provider_ID,hospital_name,city,state,county_name from hospital_general_information")
    rows = cur.fetchall()
    for row in rows:
        list_element.append(row)
        
list_element = [list(elem) for elem in list_element]


# Downloading the excel sheet and creating workbooks:
url = "http://kevincrook.com/utd/hospital_ranking_focus_states.xlsx"
r = requests.get(url)
xf = open("hospital_ranking_focus_states.xlsx","wb")
xf.write(r.content)
workbook = xlrd.open_workbook(os.path.join('hospital_ranking_focus_states.xlsx'))
sheet_1_name = workbook.sheet_by_index(0)
#sheet_1_name = workbook.get_sheet_names()[0]
list_element1 = []
for row in range(sheet_1_name.nrows):#loop for sheet
    cols = sheet_1_name.row_values(row)
    list_element1.append(cols)
sheet_2_name = workbook.sheet_by_index(1)
#sheet_2 = workbook.get_sheet_by_name(sheet_1_name)
list_element2 = []
for row in range (sheet_2_name.nrows):
    cols = sheet_2_name.row_values(row)
    list_element2.append(cols)

#Creating the Hospital Ranking MS Excel Workbook

    
list_element3 = []
for a in list_element2[:]:
    list_element3.append(a[0])
list_element_supp = list_element1[1:101]

list_data_focus_state = []
for a in list_element2[1:]:
    list_data_focus_state.append(a[1])

for i in list_element_supp:
    for j in list_element:
        if j[0] == i[0]:
            final_list.append(j)#checking for name

list_focus_state = list_element2[1:][0:]


dict_st ={}
for i in list_data_focus_state:
    state =[]
    for j in list_element:#checking for names
        if i == j[3]:
            state.append(j)
    dict_st[i] = state#appending
    
data_st_final =[]
count = 1
key ='CA'
final_dict = {}#creating dict
for st in list_data_focus_state:
    data_st_final =[]
    count = 1
    for i in list_element1:
        for j in dict_st[st]:
               if  i[0] == j[0]:
                    data_st_final.append(j)
                    count += 1
        if count > 100:
            break
    final_dict[st] = data_st_final

    

t = ['CA','FL','GA','IL','KS','MI','NY','OH','PA','TX']
#colnames = ['Provider ID','Hospital Name','City','State','County']
# creating the excel output
dest_file = 'hospital_ranking.xlsx'
workbook1 = openpyxl.Workbook()
sheet = workbook1.create_sheet("Nationwide")
sheet.cell(row = 1,column = 1,value = "Provider ID")
sheet.cell(row = 1,column = 2,value = "Hospital Name")
sheet.cell(row = 1,column = 3,value = "City")
sheet.cell(row = 1,column = 4,value = "State")
sheet.cell(row = 1,column = 5,value = "County")

for row in final_list:#looping
    sheet.append(row)


for j in list_element3:
    if j == 'State Name':#naming
        continue
    else:    
        sheet = workbook1.create_sheet(j)
        sheet.cell(row = 1,column = 1,value = "Provider ID")
        sheet.cell(row = 1,column = 2,value = "Hospital Name")
        sheet.cell(row = 1,column = 3,value = "City")
        sheet.cell(row = 1,column = 4,value = "State")
        sheet.cell(row = 1,column = 5,value = "County")
    for keys, values in final_dict.items():
        if keys[0] == j[0]:
            for row in values:
                sheet.append(row)

workbook1.remove_sheet(workbook1.get_sheet_by_name("Sheet"))
workbook1.save(dest_file)

#Create the Measures Statistical Analysis MS Excel Workbook
list_element_measure = []
cur.execute("select state,measure_id,measure_name,score from timely_and_effective_care___hospital order by measure_id")
rows = cur.fetchall()#fetching sql
for row in rows:
    list_element_measure.append(row)
list_element_measure = [list(elem) for elem in list_element_measure]#creating list

filter_measures = [element for element in list_element_measure if element[3].isnumeric()]


measures = []
for i in list_element_measure:
    measures.append(i[1])
measures1 = list(set(measures))
measures1 = sorted(measures1 ,key =  lambda x : x)
measures1

for a in filter_measures:
    a[3] = float(a[3])
#filter_measures
measure_calculations = []
for measure in measures1:#loop
    data_measure =[]
    for record in filter_measures:
        if record[1] == measure:#checking for record
            mname = record[2]
            data_measure.append(record[3])
    arr = np.array(data_measure)#checking for min max
    if arr.size > 0:
        meas_avg = np.mean(arr)
        meas_std = np.std(arr)
        meas_min = np.min(arr)
        meas_max = np.max(arr)
        #appending values
        measure_calculations.append([measure,mname,meas_min,meas_max,meas_avg,meas_std])

    

state_dict= {}
for i in list_data_focus_state:
    state_data = []
    measure_calculations_state = []#creating table
    for record in filter_measures:
        if record[0] == i:
            state_data.append(record)#appending record
    for measure in measures1:
        data_measure =[]
        for data in state_data:
            if data[1] == measure:
                mname = data[2]
                data_measure.append(data[3])
        arr = np.array(data_measure)
        if arr.size > 0:
            meas_avg = np.mean(arr)
            meas_std = np.std(arr)
            meas_min = np.min(arr)
            meas_max = np.max(arr)
            
            measure_calculations_state.append([measure,mname,meas_min,meas_max,meas_avg,meas_std])

    state_dict[i] = measure_calculations_state

#creating excel for the output
dest1_file = 'measures_statistics.xlsx'
workbook2 = openpyxl.Workbook()
sheet = workbook2.create_sheet("Nationwide")
sheet.cell(row = 1,column = 1,value = "Measure ID")
sheet.cell(row = 1,column = 2,value = "Measure Name")
sheet.cell(row = 1,column = 3,value = "Minimum")#row
sheet.cell(row = 1,column = 4,value = "Maximum")
sheet.cell(row = 1,column = 5,value = "Average")
sheet.cell(row = 1,column = 6,value = "Standard Deviation")
for row in measure_calculations:#loop 
    sheet.append(row)

for j in list_element3:
    if j == 'State Name':#sheet name
        continue
    else:    
        sheet = workbook2.create_sheet(j)
        sheet.cell(row = 1,column = 1,value = "Measure ID")
        sheet.cell(row = 1,column = 2,value = "Measure Name")#column name
        sheet.cell(row = 1,column = 3,value = "Minimum")
        sheet.cell(row = 1,column = 4,value = "Maximum")
        sheet.cell(row = 1,column = 5,value = "Average")
        sheet.cell(row = 1,column = 6,value = "Standard Deviation")
    for keys, values in state_dict.items():#loop
        if keys[0] == j[0]:
            for row in values:
                sheet.append(row)#append
    

workbook2.remove_sheet(workbook2.get_sheet_by_name("Sheet"))
workbook2.save(dest1_file)#saving excel
